In [7]:
!pip3 install langchain==0.0.319 chromadb==0.3.29 boto3 transformers python-dotenv openai accelerate optimum auto-gptq tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.4/954.4 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 112.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB

In [4]:
!wget https://github.com/mgxs-gnss/lore-ai/raw/main/chroma.zip
!mkdir chroma
!mv chroma.zip chroma
!cd ./chroma
!unzip -o ./chroma/chroma.zip

--2023-10-31 19:49:25--  https://github.com/mgxs-gnss/lore-ai/raw/main/chroma.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/mgxs-gnss/lore-ai/main/chroma.zip [following]
--2023-10-31 19:49:25--  https://media.githubusercontent.com/media/mgxs-gnss/lore-ai/main/chroma.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 383019106 (365M) [application/zip]
Saving to: ‘chroma.zip’

chroma.zip          100%[===================>] 365.28M   227MB/s    in 1.6s    

2023-10-31 19:49:36 (227 MB/s) - ‘chroma.zip’ saved [383019106/383019106]

Archive:  ./chroma/chroma.zip
  inflating: ch

In [8]:
import os
import torch
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, TextStreamer, pipeline
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.storage import InMemoryStore, LocalFileStore, RedisStore, UpstashRedisStore
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
embedding=OpenAIEmbeddings()
persist_directory='./chroma'
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embedding)
print(vectordb._collection.count())

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

26586


In [7]:
!nvidia-smi

Tue Oct 31 19:16:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
MODEL_NAME="TheBloke/Llama-2-13b-Chat-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,torch_dtype=torch.float16,trust_remote_code=True,device_map='cuda')
generation_config=GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 256
generation_config.temperature = 0.01
generation_config.do_sample=True
text_pipeline = pipeline("text-generation",model=model,tokenizer=tokenizer,return_full_text=True,generation_config=generation_config)
llm = HuggingFacePipeline(pipeline=text_pipeline)

In [13]:
template_article = """ Use the following pieces of context to formulate an article about the subject at the end. Use three sentences maximum. Keep the answer as concise as possible.
{context}
Subject: {subject}
Article:"""
Article_CHAIN_PROMPT = PromptTemplate.from_template(template_article)


article_chain = LLMChain(
    llm=llm,
    prompt=Article_CHAIN_PROMPT,
)



template_sd = """ You are a computer vision and you do scene descriptor and cataloguer, you need to identify objects that could belong to this scene. If there's none imagine which things could fit, or how it should look like.  Use 20 words maximum. Keep the answer as concise as possible.
scene: {scene}
SYSTEM:"""
sd_CHAIN_PROMPT = PromptTemplate.from_template(template_sd)

sd_chain = LLMChain(
    llm=llm,
    prompt=sd_CHAIN_PROMPT,
)

template_char = """ You are Roleplaying as a {char}, you need to rewrite the phrase so it feels like it. just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
{char} Description:
Caelia is a highly intelligent and empathetic scientist raised in a technologically advanced colony on Mars. She's driven by principles of honesty, integrity, and justice. Notably, she discovered a method to generate sustainable energy on Mars. However, she's burdened with knowledge of a major technological disaster on the colony, which was covered up by its leaders, including her own parents. She grapples with her fears of failure, overthinking, and a dependence on her family's approval, which conflict with her desire for truth and justice. Her main motivation is to expose the leadership's secrets for the betterment of Mars. As she struggles with internal and external conflicts, she evolves from a passive, doubting individual to a decisive leader. A significant relationship in her life is with her research partner, Dr. Idris Vale, who provides moral support. Throughout the story, she wrestles with her moral compass, often second-guessing herself, but ultimately becomes a beacon of resilience and courage.

Phrase: {phrase}
{char}:"""
Char_CHAIN_PROMPT = PromptTemplate.from_template(template_char)

char_chain = LLMChain(
    llm=llm,
    prompt=Char_CHAIN_PROMPT,
)


def query(subject):

    response = vectordb.max_marginal_relevance_search(subject,k=2, fetch_k=5)
    return response

def describe(subject):
    query = vectordb.max_marginal_relevance_search(subject,k=2, fetch_k=5)
    response = article_chain.run({'subject': subject, 'context':query})

    return response

def talk(subject,char, description):
    template_char = """[INST] <<SYS>>
    You are role-playing as an {char}. only short answers are allowed. Do not reveal your role, speak in the third person
    {char} Description:{description}
    context: {context}
    <<SYS>>
    {char}:"""
    Char_CHAIN_PROMPT = PromptTemplate.from_template(template_char)

    char_chain = LLMChain(
        llm=llm,
        prompt=Char_CHAIN_PROMPT,
    )

    char = char_chain.run({'context': subject, 'char':char, "description":description})
    return char

def sd(subject):
    sd = sd_chain.run({'scene': subject})
    return sd

In [18]:
describe("the seventh era")

' The seventh era of the GNSS universe, known as the Celestial Era, is marked by a sense of unity and cooperation among interstellar nations, with the Asian Empires serving as mediators and facilitators of peace. This era is characterized by advanced technology, cultural diversity, and a pursuit of knowledge, leading to an era of unprecedented collaboration and mutual understanding. The emphasis on harmony and cultural exchange becomes a guiding principle for the intergalactic society, fostering unity, innovation, and artistic expression. The Celestial Era stands as a testament to the potential of interstellar civilizations to transcend boundaries and create a harmonious and prosperous future.'